<h1>Import Models</h1>

In [1]:
import pandas as pd 
import os
import shutil
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
import PIL
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import sklearn
from sklearn import metrics

<h1>Pandas</h1>

In [3]:
dataset_path = './model2_data/classes.csv'
img_folder_train = './model2_data/allData/'
img_folder_test = './model2_data/testData/'
dataset = pd.read_csv(dataset_path)
dataset.head()

,filename,OK,spaghetti,stringing,unsmooth_surface
0,stringing_158_jpg.rf.18396c1d56a508411952d165f...,0,0,1,0
1,stringing_111_jpg.rf.18df0b478a32919a335ff2c95...,0,0,1,0
2,spaghetti_32_jpg.rf.19ef7bfb0484134b619e003086...,0,1,0,0
3,OK_37_jpg.rf.17dd174f9b97c1f4a4a13d3ba567971d.jpg,1,0,0,0
4,stringing_122_jpg.rf.1b1bef7f4adc5cfbcee2cd541...,0,0,1,0


<h1>Create Custom Dataset</h1>

In [4]:



class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, img_folder, transform) -> None:
        super().__init__()
        self.dataset = dataset
        self.img_folder = img_folder
        self.transform = transform
    def __len__(self) -> int:
        return len(self.dataset)
    def __getitem__(self, index):
        img_path = f"{img_folder + self.dataset.loc[index]['filename']}"
        label = self.dataset.loc[index]['label']
        img = PIL.Image.open(img_path)
        img = self.transform(img)
        return img, label

<h1>Building Model and Dataset</h1>

In [7]:
model = models.resnet50(pretrained=True)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset = CustomDataset(dataset, img_folder_train, transform)

test_dataset = datasets.ImageFolder(img_folder_test,transform=transform)

train_dataset, test_dataset = random_split(dataset, [int(len(dataset)*0.75), int(len(dataset)*0.25)])

/Users/interstellarobject/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/interstellarobject/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<h1>Dataloader</h1>

In [8]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


<h1>Modifying Model</h1>

In [ ]:
params = model.parameters()
for param in params:
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(2048, 512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(512, 4),
    nn.LogSoftmax(dim=1)
)


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(12):

    running_loss = 0.0
    train_loss_history = []
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 10 == 5:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
model.eval()

correct = 0
total = 0
all_labels = []
all_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        modified_tensor = labels.clone()
        modified_tensor[labels == 1] = 3
        modified_tensor[labels == 2] = 1
        modified_tensor[labels == 3] = 2
        real = modified_tensor
        print(predicted,"with",modified_tensor)
        total += labels.size(0)
        all_labels.extend(real.numpy())
        all_preds.extend(predicted.numpy())
        correct += (predicted == real).sum().item()

accuracy = correct / total * 100
print(f'Test Accuracy: {accuracy:.2f}%')
        